In [5]:
# Data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [7]:
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
# removing the top layer of the base model
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# creating the head of the model 
head_Model = baseModel.output
head_Model = MaxPooling2D(pool_size = (2, 2)) (head_Model)
head_Model = Conv2D(32,(2,2),activation='relu') (head_Model)
head_Model = AveragePooling2D(pool_size=(2, 2))(head_Model)
head_Model = Flatten(name="flatten")(head_Model)
head_Model = Dense(128, activation="relu")(head_Model)
head_Model = Dropout(0.5)(head_Model)
head_Model = Dense(2, activation="softmax")(head_Model)

In [8]:
# placing the head  model on top of the base model 
model = Model(inputs=baseModel.input, outputs=head_Model)
#looping over all layers to freeze the model
for layer in baseModel.layers:
    layer.trainable = False
# compiling the model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
      metrics=["accuracy"])

[INFO] compiling model...


D:\anaconda\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
